# Earth Observation Foundation Models: Encoder Representation Analysis and Downstream Task Benchmarking

## Section A: Introduction

---
### Scientific Background and Problem Formulation
---

TODO - can take somestuff from Ryan's previous write-up / poster 



---
### Technical Background and Introduction
---

#### Transfer Learning and Beyond

---

Transfer learning is a powerful technique in machine learning that allows a model trained on one task to leverage the previously learned features and be reused for a different but related task. Instead of starting from scratch, transfer learning leverages the knowledge a model has already gained from a separate, and usually large, dataset and applies it to a new problem. This is particularly useful when a dataset is small or annotations for the new task are limited. Once an initial training has been completed, the layers towards the front (closer to the input layer) of these models typically learn general features, such as edges in images or syntactic patterns in text, which can be useful across a wide range of tasks. Ideally, by fine-tuning the initial layers of a pre-trained model or adding new layers specific to the new task, practicioners can adapt it to perform well with far fewer training samples. This also has the potential to improve performance thresholds for a given smaller or imbalanced dataset, as the pre-trained model already has latent feature extraction capabilities from prior training.  Properly executed transfer learning not only has the potential to boost performance, but it also saves computational resources and significantly reduces training time. There are different strategies for transfer learning, such as freezing different sets of layers during training, or allowing all layers to be updated, depending on the similarity between the source (initial) and target (new) tasks. For example, in medical imaging, models trained on natural images can be adapted to detect tumors or classify X-rays by fine-tuning only a few layers.

<img src="./assets/pretraining_workflow.png" style='height:400px'/>

Figure 1. Simplified foundation model pretraining workflow

An emerging category of models coined **'Foundation Models (FM)'** subset the transfer learning technique in which a given model's pretraining task is derived from a model learning to represent, extract features, and reconstruct raw unlabelled data. This is in contrast to traditional supervised learning problems that aim to map data samples to a annotations from a specific task. In other words, these FMs are trained in a 'self-supervised' manner and potentially offer similar benefits to models developed using traditional transfer learning techniques (i.e reduced training time and performance). However, unlike with transfer learning, FMs circumvent the development of an intial large scale labelled task. This technique is especially powerful in the remote sensing world where the availability of large scale unprocessed data is an ever-increasing, and the methods for transforming it into actionable insights draw from a wide range of scientific disciplines. **The larger goal of the earth observation foundation model (EOFM) development communitity is to create models capable of generalizing across Earth Science domains and remote sensing modalities by learning from the structure and correlations iniherent in the raw data, and open the door for a wider group of practice and more versatile models that can adapt with minimal fine-tuning. Attempts towards this ephemeral goal, however, must be thoroughly assessed in a way that adheres not just to typical computer vision analysis, but also to the rigor required for the underlying science goals and that is relavent to the greater remote sensing community.** 

This notebook will cover: a simplified understanding of EOFMs, how to utilize them, evaluate their intermediate ability to extract and represent data features, benchmark their performance, and what to consider when assessing their utility for your downstream task. We assume that you have some familiarity with training deep learning model and the software associated with that process as well as a high level understanding of concepts such as network weights, back propagation, and checkpoints.

---

####  Pretraining Methodologies

---

Below are 3 common methods for pretraining a computer vision model in a self-supervised fashion. This is by no means not an exhaustive list! Due to widespread availability of satellite data, a number of variations and/or combinations of the methods below exist along with a number of alternative techniques that are not in widespread use. There is also an extensive effort to interconnect strides made separately within the computer vision and natural language processing domains both in and out of the remote sensing community. For simplicity, we omit those from this notebook. It is worth nothing that the models described and tested here are all 'transformer-based' where the core architecural unit is a vision-transformer trained under the 'encoder-decoder' paradigm (to be discussed). While it is important to understand the high level distinction between these techniques, it is not entirely necessary in order to implement them in practice as many developer groups have worked hard in abstracting most of the logic away in various frameworks. Deep understanding of each of these model families as well as the nuances of a transformer is out of the scope of this notebook but we encourage folks to learn as much as possible! The deeper a practicioner's knowledge, the easier it will be to disctinguish between appropriate and inappropriate practices both with data and the associated model training and deployment.

#### <u>Masked Autoencoding (MAE)</u> 

<img src="./assets/mae.png" style='height:400px'/>

Figure 2. Diagram depicting masked autoencoder training. Images are broken into patches that are subsequently embedded, or translated into machine-inderstandable representations of features (typically with a linear layer), and masked. The decoder is tasked with reconstructing the original image from the unmasked context.

Autoencoders consist of an encoder that projects input images into a (typically)lower-dimensional latent space and a decoder that reconstructs the original images from these representations. This compression helps the model capture essential features while discarding noise or irrelevant details. The most dominent, though not necessarily the most performant, training method is masked image modeling (i.e the 'Masked Autoencoder' (MAE)) where the model is tasked with reconstructing the masked part of an image from unmasked context. There are a handful of masking strageties that researchers have presented though they all work under the same overarching idea that a model optimized on the reconstruction can represent dataset-inherrent features well and is generalizable, or able to perform well on unseen or new data samples.

#### <u>Contrastive Learning</u> 

<img src="./assets/simclr.png" style='height:400px'/>

Figure 3. Diagram for SimCLR training paradigm.

Contrastive models are tasked with distinguishing between similar and dissimilar image representations. It works by pulling together representations of positive pairs (different augmented views of the same image) while pushing apart representations of negative pairs, which are views from different images. Common methods include 'Simple Contrastive Learning' (SimCLR) and 'Momentum Contrast' (MoCo). Unlike autoencoders, a decoder is typically not included in the pretraining phase since the task is executed entirely in the latent space. This methodology also typically requires some level of annotation - creating positive / negative pairs within the training data and can sometimes be referenced to as a 'semi-supervised' methodology.

#### <u>Non-contrastive Self Distillation</u> 

<img src="./assets/dino.png" style='height:400px'/>

Figure 4. Diagram for DINO training paradigm.

Non-contrastive self-distillation is a self-supervised learning approach for computer vision that avoids the need for negative samples. Instead of contrasting different images, it trains a student network to match the output of a teacher network, both fed with different augmented views of the same image. The teacher is often an exponential moving average of the student, providing stable targets for the two encoders. The objective is to produce consistent, meaningful embeddings, not to reconstruct or generate the input. Common methods include 'Self-**di**stillation with **no** labels' (DINO) and 'Bootstrap Your Own Latent' (BYOL).

---

#### Encoder Evaluation

---
For the evaluation of encoders, as with traditional supervised tasks, performance should not rely solely on evaluation of the objective function on training and validation subsets of the data. While this is slightly different than evaluating the performance of a model on its ultimate skill of properly mapping input samples to a target distribution, the same principals should apply - a test subset of data previously unseen to the model should be used to evaluate the model's ability to perform the task at hand in quantitative and qualitative methods. Another benefit of using encoder / decoder paradigmatic architectures the fact that the intermediate separation in the model can be used to evaluate representational skill.

While this chapter is not focused on dataset accumulation, it should also be mentioned that proper splitting of the data in a geospatial setting is imperative for proper evaluation and should consider variables like keeping unseen spatiotemporal areas for (a test set) independent post-training evaluation of a model. This ensures that complex spatiotemporal cues built into geospatial datasets are not "memorized" and overstate a model's performance on a given task and potential generalizability (LaHaye et al., 2021; Jakubik et al., 2023). Here we discuss some ways an encoder's representational "skill" can be measured and evaluated by looking at both the weight matrix of the model and the output of the encoder, or intermediate data representation called encodings or embeddings. Unlike the metrics used for downstream task evaluation, many of these are newer and therefore less known. Here we provide a brief description and references for each methodology used.

##### <u>Resource Requirements</u> 
 
Understanding the resource demands of deep learning models is crucial for both model selection and hardware planning. As data science practicioners, we typically we want to balance performance, interpretability, and resource requirements as three main driving factors in model selection. It is also crucial that we aim to choose the smallest and most efficient model that meets the requirements for our task in these three areas. This is important not only for the sake of efficiency, but also because the environmental impact of large-scale deep learning models has become a significant concern as model sizes and computational demands continue to grow. Choosing small and optimized models where appropriate is an effective strategy for reducing the emissions and energy consumption footprint associated with model development and deployment (Strubell et al., 2019; Patterson et al., 2021) . These considerations should also include a model's pretraining phase as well as resources required for operational inference.
 
 
 Three commonly used metrics Floating Point Operations (FLOPs), Multiply-Accumulate Operations (MACs), and parameter counts offer practical proxies for estimating these requirements. FLOPs denote the total number of floating-point operations (additions, multiplications, etc.) required to process a single input through the model. High FLOPs indicate a model that requires substantial computation resources, which can translate directly to longer inference and training times.MACs refer to the number of multiply-accumulate operations, foundational to neural network layers (especially in convolutions and matrix multiplications). MACs are closely linked to the actual physical operations that deep learning accelerators (like GPUs and TPUs) execute, thus providing an estimate of real computational workload. The parameter count measures the total number of learnable variables (weights and biases) are within a model. The number of parameters is a large driver for the memory required to load and store the model during execution and on disk.

Some limitations that should be considered:

1) FLOPs are platform-agnostic and do not directly account for hardware-specific optimizations or parallelization capabilities.

2) MACs are specifically meaningful for models dominated by linear algebra; other operations (e.g., activations, normalizations) may not be included in MAC counts

These parameters collectively can give us a well-rounded glimpse at model resource requirements, and allow us to factor that in when doing intercomparisons.


##### <u>Weight Matrix Analysis for a Data-Free Assessment of Training Quality</u> 

The Weight Watcher analysis library introduces the measure of an implicit form of self-regularization, as revealed by the spectral properties of a model's weight matrices (Martin et al.,  2021). The toolkit and associated papers apply Random Matrix Theory (RMT), a branch of mathematics that studies the properties of matrices whose entries are random variables and more specifically Empirical Spectral Density (ESD) analysis, the characterization of the distribution of eigenvalues of a given matrix—often large, random, or structured matrices forming a histogram that represents how the eigenvalues are distributed across their range of possible values. As a part of this analysis, the $\alpha$ parameter is an exponent characterizing the tail of the empirical spectral density (ESD) of a neural network layer’s weight matrix. When the distribution of singular values (or eigenvalues) of a weight matrix follows a power law distribution, the ESD behaves as: $\rho$($\lambda$)∼$\lambda$−$\alpha$

where $\lambda$ denotes the eigenvalues, and $\alpha$ is the power law exponent. This exponent is estimated empirically from the tail of the ESD for a given layer. $\alpha$ quantifies how "heavy-tailed" or "spread out" the layer's eigenvalue distribution is. A smaller $\alpha$ means a heavier tail: the weight matrix has more large singular values, indicating more complex/lower-rank structure. In this context, this means a given layer captures not only the true structure but also the noise—too much complexity. A larger αα means a faster-decaying tail: the singular values decay rapidly, indicating dominance of noise-like randomness or over-regularization. Here, this means a given layer fails to capture the relevant data structure—too much simplicity.

Here we provide the means to do the full ESD-based analysis that the WeightWatcher tool provides, and use the. $\alpha$ parameter as a summarization metric for the analysis.


##### <u>Working with Embeddings</u> 
TODO

##### <u>Manifold Projections for Qualitative Analysis and Dimension Reduction</u> 

The Uniform Manifold Approximation and Projection (UMAP) methodology for Dimension Reduction can be used to as both a general dimension reduction tool and as an interpretability / visualization tool, and here we use it for both. UMAP assumes data lies on a manifold and utilizes local approximations (k-nearest neighbor graphs) to represent the data’s topological and metric structure. Each local neighborhood is encoded as a fuzzy simplicial set, and then merged probabilistically to form a global topological structure (McInnes et al., 2020). Figure X provides a couple of examples of embeddings from different models and geospatial datasets projceted into a lower feature space using UMAP, and overlayed with each sample's target value. 


<img src="./assets/UMAP_Examples.png" style='height:400px'/>
Figure X. Embeddings from different models and geospatial datasets with 4 classes (left), 2 classes (center), and 7 classes (right) projected into a 2D feature space using UMAP. 

Notice that the separation of label sets in the degrades from left to right, indicating the models' increasing challenge in representing class distinctions. Note that this does not always directly correlate with the fact that a model "cannot" represent the data's structure, (although this is definitely the case some of the time) but moreso that the representation is not simple, leading to difficulties in projecting it into a lower dimension. While this is a limitation of the tool, it is useful in two ways:

1) This also likely means that downstream learning task will also be harder, as it will require more care and precision to extract representation using a given decoder head.
2) **Data needs to be human interpretable in order to be useful in the sciences, as our goal is to answer questions and understand the "whys" of processes, not just meet metric thresholds and deploy automated decision making systems.** The authors feel strongly that this should be the aim for all data science and deep learning practice, but definitely here in these domains.

Some additional limitations include: 

1) the use of stochastic operations leading to different runs or subsampling yielding varied results and
2) the quality and nature of embeddings are sensitive to hyperparameters, although empirically, the authors note that while this is true a general similarity of structure can be achieved relative to class separation if the representations of N classes are distinct enough.

The uncertainties of these tools require us to use multiple approaches to get a more comprehensive look at the state of model representation.


##### <u>Data Kernels for Inter-Model Embedding Comparisons</u> 
TODO



Like performance metrics for downstream tasks, each of these methodologies provides a piece of the full picture, and collectively they can provide a more comprehensive view of a model's representational skill and potential generalizability.


---

#### Decoders

---

You're probably wondering at this point, what do you do with the trained encoder? Just as you would with any trained neural network, you can load the weights for inference using similar code you might have seen in previous chapters. There is still one more consideration you must make depending on the type of task at hand and the shape of the output of the encoder. If, your goal is to classify images, a lightweight machine learning algorithm directly on top of the vector outputs of the encoders may be sufficient. The assumption here is that the internal representation of the image is robust and **relavent** to your task. These are the 'embeddings' you may have heard of recently and are analigous to large coarse resolution raster. We will touch on this topic in a separate chapter. If your goal, on the other hand, is to segment the images, we may need to train a subsequent neural network to extract from that internal representation and produce a useable map. Below is a summary list of contemporary decoders used in conjunction with transformer-based encoders. Each vary in complexity, purpose, and method in which they extract features from the transformer layers in the aforementioned encoders.

#### Linear Probe

<img src="./assets/linear.png" style='height:400px'/>

Figure 8. Simplified structure of a single linear layer.

The simplest decoder available. It is a single linear layer to process just the output of the encoder. In practice, this isn't typically used for segmentation due a limited model complexity. But this decoder is still useful in assessing representation quality as it is cheap to train and can be implemented in a consistently, making it an ideal tool for benchmarking. This technique is taken from the language domain where large language models are often evaluated on downstream tasks with linear probes on frozen embeddings before doing full fine-tuning. 

##### FCN

<img src="./assets/fcn.png" style='height:400px'/>

Figure 5. Full convolutional neural network architecural diagram.

The FCN is a stack of convolutional layers that upsamples and refines final feature maps produced by the transformer’s output embeddings and converts them back into an image-like spatial representation. This final representation can then be fed through a final prediction layer, which is typically itself convolutional layers, to produce the final map. This is typically more lightweight than some of the MLP-based decoders below, due to weight sharing in the learned kernel filters. Local context is also explicitly emphasized as those same kernel filters may only cover a small portion of the whole input image.

##### Segformer

<img src="./assets/segformer.png" style='height:400px'/>

Figure 6. Segformer style network architecural diagram for semantic segmentation.

Segformers use a MLP style decoder to extract from a hierarchical transformer-based encoder. Originally developed as a standalone framework for end to end training, the segformer has been recently adapted by a handful of research groups attempting to utilize the representation generated by the pretraining phase. The assumption is that the implementation of the transformer blocks in the encoder are sufficiently hierarchical in nature such that they can be composed using this method.

##### UperNet

<img src="./assets/upernet.png" style='height:400px'/>

Figure 7. UperNet architectural diagram.

Similar to the segformer architecture, the upernet leverages multi-scale hierarchical features to generate dense predictions. Rather than MLPs, the upernet utilizes pooling layers and convolutions to compose the features before the final prediction layers. It's worth noting here that the upernet was designed with a ResNet-50 backbone that was fine tuned. In other words, this closely aligns with the 'pretrained encoder' with a decoder head methology that has been popularized in the last year.

##### Muster

<img src="./assets/muster.png" style='height:400px'/>

Figure 9. Architecural diagram of MUSTER decoder.

A more recent decoder development in the computer vision realm. Similar to both the Segformer and the UperNet, hierarchical features play an important role in it's predictive strength with the most important distinction here being that the decoder itself is also transformer based. Also similar to the UperNet, the MUSTER decoder was designed to integrate with pretrained encoders. While this may lend it self to greater performance, it is important to consider the compute required to implement such an architecture.


## Section B: The Framework

---
---

[Link to original repository](https://github.com/VMarsocci/pangaea-bench)

We apply the Pangaea Bench benchmarking framework originally developed by the ESA Phi Lab team. While, there are other frameworks in development by other teams, Pangaea conveniently has many of the more popular remote sensing foundation models integrated into their pure pytorch framework along with a number of well established benchmarking datasets. It also includes a relatively simple dataset integration scheme along with additional decoder heads integrated by the team at Spatial Informatics Group for the purpose of evaluating methods for extracting the information from the internal representation of the foundation model. The limited dependency is especially beneficial for small scale demonstrations such as this learning notebook. You will find similar themes found in previous chapters of the Applied Deep Learning Book such as dataloaders, pytorch modules, loss functions, etc. These are not exclusiive to training end to end models and in fact, the workflow is almost exactly the same! The intended purpose of EOFMs is solving the same segmentation, classification, or regression problems. Here, we focus on the additional considerations that come with using an EOFM for segmentation.

<img src="./assets/pangaea_workflow.png" style='height:400px'/>

Figure 5. Diagram of Pangaea's general structure.

---

### Software/Hardware considerations

CUDA is the underlying software running NVidia GPUs. While it is not necessary, we recommend running this notebook in a CUDA enabled linux environment for convenience and setup simplicity. We also recommend installing python>=3.10 either standalone or through a conda managed environment. For running the model, we can estimate a VRAM required for storing the model using a simple calculation of # of parameters and precision. For example, a 300M parameter model at 32bit precision would roughly require 3.6GB to store the weights and optimizer states. Another 1.2 GB is required for the gradients of each of those parameters and a few more for the data itself. Likewise, the decoder used to extract from the internal representation also imposes VRAM / compute requirements. There's no universal formula here, but tools like PyTorch hooks, torch.cuda.memory_summary(), or profilers can help estimate how much compute you need. 12GB of VRAM is a good place to start and luckily there are free google colab options at this compute scale, but, there is a trend towards larger and more complex models which inevitably consume more compute. 

#### Installing framework and requirements

We are installing directly from a cloned repo along with the available requirements.txt file. This may take a few moments as PyTorch is a fairly large package. Please install python and pip on your own and setup on your notebook environment that best suits your preferences. 

In [ ]:
!git clone https://github.com/sig-gis/pangaea-bench.git "pangaea-bench"
!pip install -e "pangaea-bench"
!pip install -r pangaea-bench/requirements.txt

#### The 'torchrun' Command

The torchrun command is a utility provided by PyTorch to launch distributed training jobs across multiple processes, nodes, or GPUs. It replaces the older torch.distributed.launch and is part of the torch.distributed module. torchrun is designed to be simple and flexible, making it easier to scale up training scripts with minimal changes. At its core, torchrun sets up the environment variables necessary for distributed training, such as RANK, WORLD_SIZE, and MASTER_ADDR, and then spawns multiple processes, one per GPU or per node, depending on the configuration. This enables parallel training using frameworks like DistributedDataParallel (DDP), which helps synchronize gradients and reduce training time significantly. This will be especially useful for training larger networks however, for the purposes of this demonstration we will only use it to run on a single machine

The Pangaea benchmarking framework is a lightweight wrappper around this command to handle most of boilerplate code that comes with training a neural network. This includes establishing the training loop, calculating metrics, logging, as well as the datasets/dataloaders associated with the benchmarks. [See torchrun documentation on environment variables for more details](https://docs.pytorch.org/docs/stable/elastic/run.html). There are several command options available which can be found in ./pangaea-bench/configs. We only use a subset below in this demonstration notebook but there are many more available. Config files associated with the dataset, decoder, and encoder are associated with a PyTorch module that is instantiated by the framework based on the configurations provided.

```bash
!torchrun pangaea-bench/pangaea/run.py \                                ##### The torchrun entry command
    --config-name=train \                                               ##### configuration name which can be found in ./pangaea-bench/configs
    work_dir=checkpoints \                                              ##### the directory relative to the working directory to store checkpoint outputs
    dataset=mtbs \                                              ##### the dataset config file found in ./pangaea-bench/configs
    encoder=dofa\                                                       ##### the encoder config file found in ./pangaea-bench/configs
    decoder=seg_fcn\                                                    ##### the decoder config file found in ./pangaea-bench/configs
    preprocessing=seg_default\                                          ##### preprocssing steps associated with the task type (e.g normalizing images)
    criterion=cross_entropy \                                           ##### the loss function on which to optimize the training cycle
    task=segmentation \                                                 ##### task type (others in clude classification and regression) 
    use_wandb=true \                                                    ##### whether to use the online logging software
    task.trainer.n_epochs=16 \                                          ##### number of epochs to limit training
    task.trainer.log_interval=1 \                                       ##### logging interval of calculated metrics
    task.trainer.ckpt_interval=16 \                                     ##### how often to save checkpoints based on metrics
    encoder.encoder_weights=pretrained_models/DOFA_ViT_base_e100.pth    ##### path to pretarined weights of the FM

#### Logging and Model Examination

Weights & Biases (WandB) is a popular tool for experiment tracking, model monitoring, and collaboration in machine learning workflows. It integrates seamlessly with PyTorch, TensorFlow, Keras, and other frameworks, allowing users to log training metrics, visualize results in real-time, and compare model performance across runs. Pangaea benchmark happens to have a prebaked integration of wandb which is easily accessed using WandB's authentication protocol. [See WandB documentation on environment variables for more details](https://docs.wandb.ai/guides/track/environment-variables/)

## Section C: Example Workflow 1 - Burn Scar Mapping

### Frozen vs Unfrozen vs Random vs LoRA

Testing frozen, unfrozen, and randomly initialized pretrained encoders is useful in understanding the value and applicability of transfer learning for a specific task. A frozen encoder uses pretrained weights without updating them during training. This setup helps evaluate how useful the pretrained features are on their own, especially in cases where the target dataset is small or the model is prone to overfitting. In contrast, an unfrozen encoder allows those pretrained weights to be updated, enabling the model to adapt its learned representations to better suit the target task. This approach often yields better performance when sufficient labeled data is available and the task deviates from the original pretraining domain however, requires that additional gradients be computed and stored. On the other hand, a randomly initialized encoder serves as a baseline, providing a measure of how well the model performs without any prior knowledge. Comparing results from this setup against those using pretrained weights helps quantify the benefits of pretraining in terms of training time and overall final performance. It's worth noting that depending on the task and the amount of available training data, the randomly initialized encoder may never achieve the same results as an unfrozen pretrained model. This behavior is still an on going area of research. 
Overall, these three basic configurations allow for a comprehensive assessment of whether transfer learning is useful, whether fine-tuning improves results, and whether pretraining is necessary at all for a specific task. There is a fourth method which we will not use here called Low Rank Adaption (LoRA) and is somewhat of a compromise between a fully frozen and unfrozen encoder. For larger models (i.e greater than 300m parameters). Low-Rank Adaptation (LoRA) is a technique used to efficiently fine-tune large pretrained neural networks by injecting small, trainable weight matrices into the model, while keeping the original weights frozen. Traditional fine-tuning updates all parameters of a model, which becomes expensive for large models. LoRA avoids this by approximating the weight updates as the product of two low-rank matrices, significantly reducing the number of trainable parameters. This low-rank decomposition acts as a bottleneck, enforcing efficiency and reducing overfitting. LoRA has gained popularity particularly with large language models. It also enables modular fine-tuning where separate LoRA modules can be swapped in for different tasks making it attractive for applications requiring task specialization without retraining the entire model.

TODO - some dataset-specific information here would be good


### Lets Fine Tune!

Okay, we've established some of the basic concepts, lets train some models! We'll be focusing on a single task for 3 frozen encoder variations: DOFA, CROMA, and Prithvi 1.0 (described below). Along with those 3 FMs we train the 4 different decoders described in section A. That's 12 different models for just a single task! It's clear here that for a comparison analysis for foundation models, compute becomes a key bottleneck in research especially when aiming for systematic comparisons across models, tasks, or training regimes. For the sake of this demonstration, we limit each training run to a batch size of 8 for 16 epochs an on a single RTX 4000 workstation GPU with 12GB of VRAM. A machine of this size is readily available on any cloud compute platform.

The task we are targeting is segmenting burn scars in HLS scenes. An HLS-based benchmark is unique in that it sources its optical data from two satellite missions (Landsat and Sentinel) and in doing so, provides a relatively level test bed for models trained on either. Nevertheless, it’s important to acknowledge the nature of the benchmark dataset used. While it has been carefully curated to support robust model evaluation, it may not fully reflect the diversity or complexity of real-world fire scenarios with potentially with drastically different landscape dynamics, data availability, and task scope. The dataset is limited in geographic scope, vegetation types, and imaging conditions, which means performance metrics obtained here may not generalize well to all fire contexts such as risk mapping. As such, results from this benchmark should be interpreted with consideration of these constraints.

### Highlighted Models
For the encoder evaluation, we will take a look at a large set of pretrained encoders available within Pangaea-Bench. From there, we will focus fine-tuning on a single task for 3 frozen encoder variations: DOFA, CROMA, and Prithvi 1.0 (described below). Below is a brief introduction to the models we will focus on in the fine tuning process and beyond. 
#### DOFA
[Link to paper](https://arxiv.org/pdf/2403.15356)

DOFA is a transformer-based model that is optimized on both a mask image modeling objective and self distillation objective (Figure 2b). DOFA's major contribution is their wavelength-conditioned dynamic patch embedding in which a the central wavelength of a given sensor is used to derive the weights for processing the resepective data. It is apparent from this that their approach is targettiing flexibility and generalizability across multiple modalities and does so fairly well. And yet, there are still open questions! For example, wavelength and reflectance does not necessarily translate to amplitude data from SAR. Does this matter? Benchmarking metrics will tell you no but do keep in mind: why?

<img src="./assets/dofa.png"/>

Figure 6: (a) Architecture design. DOFA builds on masked image modeling by processing input images with any number of channels within a single framework. (b) Dynamic
weight generator and continual training framework.

#### Prithvi 1.0
[Link to paper](https://arxiv.org/pdf/2310.18660)

One of the earliest foundation models and the simplest of the three models tested in this demonstration. Developed by the NASA IMPACT, Prithvi 1.0 is a vanilla masked autoencoder specifically trained exclusively on HLS data. This limited scope presents a unique research opportunity for studying a vision transformer with relatively few modifications to its structure. The expectation then is that for this specific test, Prithvi would have the best performance metrics given that its pretraining dataset more closely aligns with the downstream task. TBD!


<img src="./assets/prithvi.png"/>

Figure 7: Masked autoencoder diagram for Prithvi's pretraining.

#### CROMA
[Link to paper](https://arxiv.org/pdf/2311.00566)

CROMA is a another transformer-based model with a slightly different approach combining both masked image modeling (MIM) and contrastive learning. Rather than contrasting positive and negative samples, the CROMA framework contrasts optical and radar data using separate encoders then combines those embeddings using cross attention with a unifying transformer encoder. The output of this terminal encoder is then randomly masked and trained in a typical encoder-decoder MIM style. The optical encoder requires all 12 spectral bands from Sentinel 2 while the radar encoder requires the 2 polarization bands from Sentinel 1. In our downstream task, we are limited to 6 harmonized spectral bands. How can we expect this to impact our results?

<img src="./assets/chroma.png"/>

Figure 8: (Left) Pretraining framework for CROMA. (Right) Encoding workflow for leveraging internal representations.

#### Inter-Model Embedding Comparisons Using Data Kernels

#### Resource Requirements - FLOPS, MACS, and Paremeter Counts

#### Encoder Weight-Matrix Analysis

Before we look at the encoders relative to the input data, lets evaluate TODO 

Below is the example command for CROMA. The commands for all other encoders can be found here: [Multi-Encoder Per-Layer Weight Matrix Analysis](https://github.com/sig-gis/pangaea-bench/blob/embedding_analysis_nl/scripts/encoder_analysis/ww_mtbs.sh)

In [ ]:
#Move to directory.
!pushd pangaea-bench/pangaea/encoder_analysis/
#Library writes to local directories.
!mkdir img
!mkdir ww-img
!python3 weight_watcher.py dataset=mtbs encoder=croma_optical \
    preprocessing=seg_resize_input_layer task=segmentation
!mkdir ww/croma/
!mv img/* ww/croma/
!mv ww-img/* ww/croma/

There are further details produced by the commands run, but we will use the per-layer $\alpha$ plots in Figure X to provide a top-level summary of the findings. CROMA appears to have the most consistently well-trained layers, with most of the $\alpha$ values staying within the desired [2,5] range, and the deviation outside of this range being very minimal. Prithvi appears to have a handful of layers that are overfit to a small degree, and Dofa has a somewhat evenly distributed set of layers throughout the model that appear to be underfit. 

All models seem to be following a phenomena observed by (Martin et al., 2021) where vision models initial layers tend to overfit. The current hypothesis is that this is somewhat beneficial to performance, and acceptable in this case, as most high-level features learned in early layers of vision models are similar across all scenes in a large training set, so the early layers in a model, typically the ones that learn these high-level features, tend towards overfitting / memorization of high-level features, and therefore a lack of generalizability. 

<img src="./assets/croma_optical_fnl_alpha_depth.png"/>
<img src="./assets/Prithvi_fnl_alpha_depth.png"/>
<img src="./assets/dofa_encoder_fnl_alpha_depth.png"/>


Figure X. Plots containing the $\alpha$ summary parameter from the model weight matrix analysis done for CROMA (top), Prithvi (center), and Dofa (bottom).


These insights not only inform us about performance considerations when using these models with frozen weights, but also can provide some insight into considerations we need to take into account if we are to fine-tune the weights of these encoders. For example: 

**1)** Which models' overfit layers might be more prone to a phenomena called catastrophic forgetting, where performance significantly dips after fine-tuning due to the information gained in pre-training being lost or overwritten.

**2)** Which model's layers may need more time to learn / optimize during the fine-tuning phase, due to being underfit in pre-training.

#### Embedding generation

Next will take the input dataset and pass it through the pre-trained encoders to get model-specific representations, or embeddings for each input scene. Below is an example command for the CROMA encoder. We also generate these for 16 other encoders. Both pre-trained and randomly initialized. Additional exampled can be found here: [MTBS Multi-Encoder Embedding Generation Script](https://github.com/sig-gis/pangaea-bench/blob/embedding_analysis_nl/scripts/encoder_analysis/embed_gen_mtbs.sh)

In [ ]:
!python3 embed.py dataset=mtbs encoder=croma_optical preprocessing=seg_default task=segmentation

#### UMAP Projection and KNN Graph Generation

Now that we have the embeddings we can project them onto a lower-dimension manifold and generate a KNN-Graph from that represenation. Below is the example command for CROMA. The commands for all other encoders can be found here: [MTBS Multi-Encoder UMAP Projection and KNN-Graph Generation](https://github.com/sig-gis/pangaea-bench/blob/embedding_analysis_nl/scripts/encoder_analysis/umap_and_knn_graph_gen_mtbs.sh)

In [ ]:
!python3 knn_graph_gen.py dataset=mtbs \
          encoder=croma_optical preprocessing=seg_default \
          criterion=cross_entropy    task=segmentation

<img src="./assets/MTBS_UMAP.png"/>
Figure X. A plot of the embeddings generated from CROMA (left), Dofa (center), and Pritvhi (left) for samples from the test split of the MTBS dataset projected onto a low-dimension (2D) manifold via UMAP and overlayed with their associated target value. The key for the targets is in the top left.


We can use these plots to first do a qualitative analysis. If we compare these UMAP plots to those in Figure X, where we can clearly see strong separations of the classes for all models, we can take away the fact that this is not an easy task for these models. In all three cases, to varying degrees, there is an intermixing of the label values.  Both Prithvi and CROMA appear to do a better job of separating out subsets of the classes in different ways, while Dofa appears to have the hardest time representing the classes distincly in its embedding space, as demonstrated by a lack of clear separation of the classes anywhere in the plot.

As mentioned before, this methodology is not perfect, and we are removing representative information from embedded samples by reducing the feature space, but trade-offs have to be made for interpretability and analysis of model performance, as results need to be human-interpretable in order for them to be useful. If a model is more easily representing distinctions between samples in a simplified way at a higher dimension, those distinctions will likely be picked up in this lower dimension space. 

#### Inter-Model Embedding Comparisons Using Data Kernels

TODO

### Lets Fine Tune!

Okay, we've established some of the basic concepts, lets train some models! Along with the 3 FMs we will train the 4 different decoders described in section A. That's 12 different models for just a single task! It's clear here that for a comparison analysis for foundation models, compute becomes a key bottleneck in research especially when aiming for systematic comparisons across models, tasks, or training regimes. For the sake of this demonstration, we limit each training run to a batch size of 8 for 16 epochs an on a single RTX 4000 workstation GPU with 12GB of VRAM. A machine of this size is readily available on any cloud compute platform.

The task we are targeting is segementing burn scars in HLS scenes. An HLS-based benchmark is unique in that it sources its optical data from two satellite missions (Landsat and Sentinel) and in doing so, provides a relatively level test bed for models trained on either. Nevertheless, it’s important to acknowledge the nature of the benchmark dataset used. While it has been carefully curated to support robust model evaluation, it may not fully reflect the diversity or complexity of real-world fire scenarios with potentially with drastically different landscape dynamics, data availability, and task scope. The dataset is limited in geographic scope, vegetation types, and imaging conditions, which means performance metrics obtained here may not generalize well to all fire contexts such as risk mapping. As such, results from this benchmark should be interpreted with consideration of these constraints.

#### DOFA


In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=dofa\
    decoder=seg_muster\
    preprocessing=seg_default\
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/DOFA_ViT_base_e100.pth

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=dofa\
    decoder=seg_fcn\
    preprocessing=seg_default\
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/DOFA_ViT_base_e100.pth

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=dofa\
    decoder=seg_linear\
    preprocessing=seg_default\
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/DOFA_ViT_base_e100.pth
    

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=dofa\
    decoder=seg_segformer\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/DOFA_ViT_base_e100.pth

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=dofa\
    decoder=seg_upernet\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/DOFA_ViT_base_e100.pth

#### Prithvi 1.0


In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=prithvi \
    decoder=seg_muster\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/Prithvi_EO_V1_100M.pt

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=prithvi \
    decoder=seg_fcn\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/Prithvi_EO_V1_100M.pt

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=prithvi \
    decoder=seg_linear\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/Prithvi_EO_V1_100M.pt


In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=prithvi\
    decoder=seg_segformer\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/Prithvi_EO_V1_100M.pt
  

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=prithvi \
    decoder=seg_upernet\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/Prithvi_EO_V1_100M.pt
    

#### CROMA
[Link to paper](https://arxiv.org/pdf/2311.00566)

CROMA is a another transformer-based model with a slightly different approach combining both masked image modeling (MIM) and contrastive learning. Rather than contrasting positive and negative samples, the CROMA framework contrasts optical and radar data using separate encoders then combines those embeddings using cross attention with a unifying transformer encoder. The output of this terminal encoder is then randomly masked and trained in a typical encoder-decoder MIM style. The optical encoder requires all 12 spectral bands from Sentinel 2 while the radar encoder requires the 2 polarization bands from Sentinel 1. In our downstream task, we are limited to 6 harmonized spectral bands. How can we expect this to impact our results?

<img src="./assets/chroma.png"/>

Figure 8: (Left) Pretraining framework for CROMA. (Right) Encoding workflow for leveraging internal representations.

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=croma_optical\
    decoder=seg_muster\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt
    

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=croma_optical\
    decoder=seg_fcn\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=croma_optical\
    decoder=seg_linear\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt


In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=croma_optical\
    decoder=seg_segformer\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt


In [ ]:
!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=hlsburnscars \
    encoder=croma_optical\
    decoder=seg_upernet\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

### Results

Some things to consider:

Below are the results from running the above commands aggregated into relatively simple charts. Keep in mind, we only train for 16 epochs for the purpose of a simple demonstration. This is a very small amount of time for a deep learning model. There is a reasonable chance that these metrics will change given enough training time, a different loss function, or a different set of hyperparameters. The framework allows for some level of configuration. On your own, try setting new configurations (see ./pangaea-bench/configs for details).

#### Training Graphs

In spite of a limited training window, we see a fairly distinct training curve which suggests the models have learned some useful features. Whether this is due to the pretraining or the simple nature of the task is still an open question. A straightforward test for that is to train using a randomly initialized encoder simply by adjusting the last configuration in each training command to null. In addition, there is a clear loss difference between each model with DOFA generally having higher cross entropy loss.

<img src="./assets/training_graph.png" style='height:600px'/>

Figure 9: Training loss per step for all model varations. Each model was optimized on cross entropy loss.

#### Decoder Choice

The test metrics for the checkpoint saved at the final epoch tells a slightly different story. Overall, the CROMA combinations appear to be the most effective in terms of overall performance metrics (F1 and IoU). The PRITHVI encoder consistently shows good performance across different decoders, with the Linear Probe surprisingly being the top performer in this group. The DOFA Linear Probe combination stands out as an outlier with extremely low performance for burn scar detection. The lower performance of the probe into DOFA suggests their internal representatioin may not be appropriate for this task and the decoder might be doing the bulk of the heavy lifting in the other runs. More experimentation is required to confirm this. It's worth noting that for even with the same encoder, varying the decoder has a visible affect on performance that is comparible in magnitude to the differences between encoders.

<img src="./assets/test_metrics.png" style='height:600px'/>

Figure 10: Test metrics per encoder-decoder variation

#### Per Class Metrics

Looking at per class metrics gives a bit more insight into what's happening. Save for the DOFA-linear-probe outlier, the differences from model to model is realistically negligible (statistical test would be recommended to confirm this). However, when we look at per class metrics, we see a more noticeable difference between each model's ability to segment the positive class. This suggests a class imbalance issue. While this may be an accurate representation of the task, this is something that needs to be taken into account when designing a benchmark dataset and evaluation framework.

<img src="./assets/per_class.png" style='height:600px'/>

Figure 11: Per class metrics

#### Run Times and Compute Costs

Based on the above metrics alone, CROMA seems to be a top performer, nevertheless, we must consider one more aspect: compute. CROMA consumed up to ~3 times the memory and up to ~9 times the training time than some of the other models for the same number of epochs. In other words, for the same resources, DOFA combined with an even larger decoder trained for 128 epochs instead of 16 may very well outperform the CROMA models. In the same vein, PRITHVI consumed a comparable amount of memory to CROMA but is far simpler and to implement and fine tunes several times faster. 

<img src="./assets/memory.png" style='height:400px'/><img src="./assets/training_time.png" style='height:400px'/>

Figure 12: (Left) GPU VRAM usage per encoder-decoder combination. (Right) Run time in seconds to reach 16 epochs.

#### Output Masks

We have all these metrics and they all tell a similar story. Lets look at some of the outputs! Afterall, the end product should be a map. The first thing you should notice below is that the annotation is not perfect! So the metrics above should be interpreted carefully. For example, the prithvi masks from all decoders look vastly different while the metrics say otherwise. When the benchmark dataset was created, chip labels that contained multiple annotations in close proximity were likely left unfiltered. This does offer an opportunity to see how a model behaves in such situations which are frankly, all too common. Interestingly whether or not the model captures the false negatives depends mostly on the decoder. In addtion, there are artifacts in the FCN variations that arise from the structure of the decoder itself rather than anything meaningful from the image.

<img src="./assets/masks.png" style='height:600px'/>

<img src="./assets/hls_label.png" style='height:400px'/> <img src="./assets/hls_image.png" style='height:400px'/>

Figure 13: (Top) Grid table of mask outputs of all models with all decoders. (Bottom Left) Original annotation. (Bottom Right) Source Image



#### Multimodality / Multisensor Models Scale

You may have noticed in spite being a small decoder, the linear probe into PRITHVI had comparable performance to the larger models in this given task. While this may be attributed to the limited training time or architectural design choices, the more likely reason is that PRITHVI 1.0 was trained exclusively on HLS scenes. Foundation Models, like all machine learning models, are shaped by their source data and making them truly generalizabile is an ongoing problem. Likewise, spatiotemporal patterns are diverse in both downstream tasks and sensor data. Ultimately when selecting a base model, one of the first things to consider before benchmarking metrics is how well does the pretraining dataset align with your given task.

## Section D: Example Workflow 2 - Burn Intensity Mapping

### A Well Defined Problem

Benchmarks such as ImageNet for vision tasks or GLUE for language tasks have helped standardize comparisons, but emerging domains may lack such comprehensive baselines. In such cases, creating task-specific benchmarks becomes more necessary especially when these "Foundation Models" become more and more prominent. Like all standardized evaluations, benchmarks can often promote excessive metric fixation. Even so, these metrics are necessary when comparing across models and must be done under the same conditions: using the same dataset splits, preprocessing steps, and evaluation protocols. Differences in any of these factors can distort the interpretation of results. For that reason, we see testing in multiple controlled conditions an essential step for a robust evaluation. In our simplified example above, we vary the decoder attached to each frozen pretrained encoder as the quality of the internal representation is only as good as how easily we can extract from it. Model complexity, training time, and inference speed are also crucial metrics to consider, especially in deployment scenarios where computational resources or latency requirements are constrained.

A well defined benchmark must incorperate all of these factors while also driving innovation. Burn scar mapping has been achieved using common remote sensing techniques with vastly smaller scale compute requirements. Lets look at a related but more difficult problem: burn intensity mapping ([see here](https://www.mtbs.gov/) for more details). Categorizing each pixel for all burns is a non-trivial task and the dNBR thresholding while powerful, has some drawbacks, namely, the time frame with clear enough imagery to make a meaningful assessment.


### Training Scripts

We'll be running the exact same commands as before with only one difference. We'll setting the dataset configuration from hlsburnscars to mtbs. That's it! Luckily for us we've also implemented the dataloader and configurations found in ./pangaea-bench/pangaea and ./pangaea-bench/configs respectively. Since this is a much larger dataset (5 times in fact), these runs may take up to two hours to run rather than a few minutes. If compute is a limitation, go ahead and skip to the results. 

In [ ]:

## CROMA

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=croma_optical\
    decoder=seg_muster\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=croma_optical\
    decoder=seg_fcn\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=croma_optical\
    decoder=seg_linear\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=croma_optical\
    decoder=seg_segformer\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=croma_optical\
    decoder=seg_upernet\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

## PRITHVI

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=prithvi\
    decoder=seg_muster\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=prithvi\
    decoder=seg_fcn\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=prithvi\
    decoder=seg_linear\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=prithvi\
    decoder=seg_segformer\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=prithvi\
    decoder=seg_upernet\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

## DOFA

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=mtbs \
    encoder=dofa \
    decoder=seg_muster\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=dofa \
    encoder=prithvi\
    decoder=seg_fcn\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=dofa \
    encoder=prithvi\
    decoder=seg_linear\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=dofa \
    encoder=prithvi\
    decoder=seg_segformer\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

!torchrun pangaea-bench/pangaea/run.py \
    --config-name=train \
    work_dir=checkpoints \
    dataset=dofa \
    encoder=prithvi\
    decoder=seg_upernet\
    preprocessing=seg_default \
    criterion=cross_entropy \
    task=segmentation \
    use_wandb=true \
    task.trainer.n_epochs=16 \
    task.trainer.log_interval=1 \
    task.trainer.ckpt_interval=16 \
    encoder.encoder_weights=pretrained_models/CROMA_large.pt

### Results

#### The Loss Graph

Figure 14: Loss graphs for all training runs.

#### Per Class Metrics

Figure 15: Best metrics per best decoder with each encoder.

#### Interpretation

<img src="./assets/boulderlake1.png" style='height:225px'/><img src="./assets/boulderlake2.png" style='height:225px'/><img src="./assets/boulderlake3.png" style='height:225px'/><img src="./assets/boulderlake4.png" style='height:225px'/>

<img src="./assets/boulderlake4wlabels.png" style='height:400px'/>

Figure 16: (TOP) Landsat imagery from the Boulder Lake, WA fire in August 2022. from Left to right: 1 year prefire,  2 months post fire, 1 year post fire, 2 year post fire. (Bottom) 2 year post fire with class labels overlaid.


## References

TODO - fix numbering, formatting, and make alphabetic

1) Jakubik, J., Roy, S., Phillips, C. E., Fraccaro, P., Godwin, D., Zadrozny, B., Szwarcman, D., Gomes, C., Nyirjesy, G., Edwards, B., Kimura, D., Simumba, N., Chu, L., Mukkavilli, S. K., Lambhate, D., Das, K., Bangalore, R., Oliveira, D., Muszynski, M., . . . Ramachandran, R. (2023b, October 28). Foundation Models for Generalist Geospatial Artificial Intelligence. arXiv.org. https://arxiv.org/abs/2310.18660

1) LaHaye, N., Garay, M. J., Bue, B. D., El-Askary, H., & Linstead, E. (2021). A Quantitative Validation of Multi-Modal Image Fusion and Segmentation for Object Detection and Tracking. Remote Sensing, 13(12), 2364. https://doi.org/10.3390/rs13122364 

2)  Charles H. Martin, Michael W. Mahoney; Implicit Self-Regularization in Deep Neural Networks: Evidence from Random Matrix Theory and Implications for Learning. JMLR 22(165):1−73, 2021 

3) Charles H. Martin, Tongsu (Serena) Peng & Michael W. Mahoney; Predicting trends in the quality of state-of-the-art neural networks without access to training or testing data. Nature Communications 12(4122), 2021 

4) McInnes, Leland, Healy, John, & Melville, James. "UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction." arXiv preprint arXiv:1802.03426, 2020

5) Patterson, D., Gonzalez, J., Le, Q., Liang, C., Munguia, L., Rothchild, D., So, D., Texier, M., & Dean, J. (2021, April 21). Carbon emissions and large neural network training. arXiv.org. https://arxiv.org/abs/2104.10350

6) Strubell, E., Ganesh, A., & McCallum, A. (2019, June 5). Energy and policy considerations for deep learning in NLP. arXiv.org. https://arxiv.org/abs/1906.02243
